# Agente: Ejemplo de un agente que hace uso de herramientas

## Instalar las librerías necesarias


In [ ]:
!pip install langchain langchain-groq langchain-experimental duckduckgo-search


## Importar las librerías necesarias

In [1]:
import os
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import Tool
from langchain_groq import ChatGroq
from langchain.agents import create_react_agent
from langchain.agents.agent import AgentExecutor
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool

## Cargar variables de entorno


In [2]:
load_dotenv()

True

## Definir el LLM

In [3]:
groq_llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.2-90b-vision-preview",
    temperature=0,
)

## Inicializar las herramientas

In [4]:
python_repl = PythonREPL()
search = DuckDuckGoSearchRun()

## Crear la herramienta con el REPL para el agente

In [5]:
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

## Crear la herramienta de búsqueda para el agente

In [6]:
search_tool = Tool(
    name="search",
    description="Search the web for information",
    func=search.run,
)

## Creamos una herramienta personalizada para hacer poemas


In [26]:
@tool
def make_a_poem(topic: str) -> str:
    """Make a poem about a topic."""
    prompt = f"Make a poem about {topic}"
    response = groq_llm.invoke(prompt)
    return response.content


## Importamos el prompt
Usamos el prompt pre-definido para el agente react

In [8]:
prompt = hub.pull("hwchase17/react")

## Creamos el agente react


In [27]:
agent = create_react_agent(
    llm=groq_llm,
    tools=[repl_tool, search_tool, make_a_poem],
    prompt=prompt,
)

## Creamos el executor del agente


In [28]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[repl_tool, search_tool, make_a_poem],
    max_iterations=5,
    handle_parsing_errors=True,
    verbose=True
)

## Función para ejecutar el agente


In [29]:
def run_repl_agent(query):
    response = agent_executor.invoke({"input": query})
    return response["output"]

## Probamos el agente


In [30]:
response = run_repl_agent("love")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to understand what the question is asking about "love". Is it a definition, a poem, or something else?

Action: make_a_poem
Action Input: loveSoft whispers in the night,
A gentle touch, a loving light,
Two hearts beating as one soul,
In love's warm flame, they make their goal.

With every kiss, a love so true,
Their hearts entwined, forever new,
In each other's eyes, they see,
A love so strong, a love so free.

The world outside may fade away,
But in each other's love, they'll stay,
Through laughter and through tears,
Their love will last, through all the years.

In love's sweet garden, they'll roam,
Hand in hand, a love to call home,
Their hearts forever entwined,
A love so pure, a love so kind.

So let love be the guiding light,
That shines so bright, in the dark of night,
For in love's warm and loving glow,
Two hearts can beat, as one, forever to grow.Thought: The poem I generated is a beautiful representation of love, but I st